# 🤟 Sign Language Translator - VERSIÓN COMPLETA

## ✨ Características Completas
- 📹 **Webcam en Gradio** (dentro de la interfaz)
- 🎥 **Captura múltiple** con detección de manos
- 🎓 **Entrenamiento** con Bi-LSTM + Attention
- 🔮 **Traducción** en tiempo real
- 📊 **240 Features** por frame
- 🔄 **Data Augmentation**

---

## 📋 Flujo Completo

### 1. Captura (Tab 1)
- Webcam en vivo en Gradio
- Click "Preview" → Ver mano + keypoints
- Click "Capturar" → Graba múltiples muestras
- Auto-pausa sin manos

### 2. Entrenamiento (Tab 2)
- Carga datos capturados
- Entrena Bi-LSTM + Attention
- Data augmentation automático
- Guarda mejor modelo

### 3. Traducción (Tab 3)
- Carga modelo entrenado
- Webcam en tiempo real
- Traduce señas instantáneamente
- Muestra confianza

### 4. Dataset Info (Tab 4)
- Estadísticas de datos
- Muestras por seña
- Recomendaciones

---
# 🔧 CELDA 1: Setup

In [ ]:
print("="*70)
print("🤟 Sign Language Translator - Setup")
print("="*70)

# 1. Montar Drive
print("\n📁 Paso 1/6: Montando Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Drive montado")

# 2. Clonar repo
print("\n📥 Paso 2/6: Clonando repositorio...")
import os
os.chdir('/content')

GITHUB_REPO = "https://github.com/Hernan-Mej/sign-language-translator.git"

if os.path.exists('sign-language-translator'):
    !cd sign-language-translator && git pull
else:
    !git clone {GITHUB_REPO}
print("✅ Repositorio listo")

# 3. Instalar deps
print("\n📦 Paso 3/6: Instalando dependencias...")
!pip install -q protobuf==4.25.8
!pip install -q -r /content/sign-language-translator/requirements.txt
print("✅ Dependencias instaladas")

# 4. Paths
print("\n🔧 Paso 4/6: Configurando paths...")
import sys
sys.path.insert(0, '/content/sign-language-translator/src')
print("✅ Paths configurados")

# 5. Estructura Drive
print("\n📂 Paso 5/6: Creando estructura...")
from pathlib import Path

drive_root = Path('/content/drive/MyDrive')
project_dir = drive_root / 'SignLanguageTranslator'

for directory in [
    project_dir / 'data' / 'raw',
    project_dir / 'data' / 'processed',
    project_dir / 'models',
    project_dir / 'logs',
]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Estructura creada")

# 6. Verificar imports
print("\n🧪 Paso 6/6: Verificando módulos...")
try:
    from config import MODEL_CONFIG, DATA_DIR, MODELS_DIR
    from key_points_extractor import KeyPointsExtractor
    from data_augmentation import SignLanguageAugmenter
    print("✅ Módulos básicos OK")
    
    # Intentar importar training
    try:
        from advanced_lstm_model import create_model
        from training import Trainer
        print("✅ Módulos de entrenamiento OK")
    except ImportError:
        print("⚠️  Módulos de entrenamiento no encontrados")
        print("   Para entrenar, agrega advanced_lstm_model.py y training.py")
except ImportError as e:
    print(f"❌ Error: {e}")

print("\n" + "="*70)
print("✅ SETUP COMPLETADO")
print("="*70)
print("\n📌 Ejecuta la celda de UI (abajo)")

---
# 🎨 CELDA 2: Interfaz Completa

In [ ]:
# ============================================================================
# INTERFAZ CORREGIDA - PROCESAMIENTO EN TIEMPO REAL UNIFICADO
# ============================================================================
# Reemplaza la celda de UI completa con este código

import gradio as gr
import cv2
import numpy as np
import json
import time
from pathlib import Path
from datetime import datetime
import tensorflow as tf

from config import DATA_DIR, MODELS_DIR, MODEL_CONFIG
from key_points_extractor import KeyPointsExtractor
from data_augmentation import SignLanguageAugmenter

# Intentar importar training
try:
    from advanced_lstm_model import create_model
    from training import Trainer
    TRAINING_AVAILABLE = True
except ImportError:
    TRAINING_AVAILABLE = False

# ============================================================================
# ESTADO GLOBAL
# ============================================================================

class AppState:
    def __init__(self):
        self.extractor = KeyPointsExtractor()
        self.augmenter = SignLanguageAugmenter()
        self.current_model = None
        self.sign_map = {}
        
        # Estado de captura
        self.capturing = False
        self.current_sign = ""
        self.total_samples = 0
        self.samples_captured = 0
        self.frames_per_sample = 30
        self.current_sequence = []
        
        # Buffer para traducción
        self.translation_buffer = []

state = AppState()

# ============================================================================
# PROCESAMIENTO UNIFICADO
# ============================================================================

def process_frame_unified(frame, mode="preview"):
    """
    Procesa frame y retorna imagen con keypoints + info.
    
    Args:
        frame: Frame de webcam (RGB)
        mode: "preview", "capture", "translate"
    
    Returns:
        processed_frame: Imagen con keypoints dibujados
        info_text: Texto informativo
    """
    if frame is None:
        return None, "⚠️ No hay frame"
    
    # Convertir a BGR para OpenCV
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Extraer keypoints (esto ya dibuja los keypoints en el frame)
    processed_frame, features = state.extractor.extract_keypoints(frame_bgr)
    
    # Detectar mano
    hand_detected = not np.all(features == 0)
    non_zero = np.count_nonzero(features)
    
    # Convertir de vuelta a RGB
    processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
    
    # Generar info según modo
    if mode == "preview":
        status = "✅ Mano detectada" if hand_detected else "⚠️ No se detecta mano"
        info = f"{status}\\n📊 Features: {non_zero}/240"
        
    elif mode == "capture":
        if not state.capturing:
            info = "⏹️ No hay captura activa\\n\\nClick 'Iniciar Captura' para empezar"
        else:
            info = f"📹 Capturando: {state.current_sign}\\n"
            info += f"📊 Muestra: {state.samples_captured + 1}/{state.total_samples}\\n"
            info += f"📐 Frames: {len(state.current_sequence)}/{state.frames_per_sample}\\n\\n"
            
            if hand_detected:
                # Agregar frame a secuencia
                state.current_sequence.append(features)
                info += "✅ Grabando...\\n"
                
                # Completar muestra
                if len(state.current_sequence) >= state.frames_per_sample:
                    sign_dir = DATA_DIR / "raw" / state.current_sign
                    sign_dir.mkdir(parents=True, exist_ok=True)
                    
                    existing = len(list(sign_dir.glob("*.npy")))
                    sequence_array = np.array(state.current_sequence[:state.frames_per_sample])
                    
                    sample_file = sign_dir / f"sample_{existing:03d}.npy"
                    np.save(sample_file, sequence_array)
                    
                    state.samples_captured += 1
                    state.current_sequence = []
                    
                    info += f"✅ Muestra {state.samples_captured} guardada!\\n"
                    
                    if state.samples_captured >= state.total_samples:
                        state.capturing = False
                        info += f"\\n🎉 COMPLETADO!\\n"
                        info += f"📦 {state.samples_captured} muestras en {sign_dir}\\n"
                    else:
                        info += f"\\n⏸️ Pausa 1 seg - Cambia ángulo\\n"
                        time.sleep(1)
            else:
                info += "⏸️ Esperando mano...\\n"
    
    elif mode == "translate":
        if state.current_model is None:
            info = "⚠️ Carga un modelo primero"
        elif not hand_detected:
            info = "⏸️ No se detecta mano"
            state.translation_buffer = []
        else:
            # Acumular frames
            state.translation_buffer.append(features)
            
            if len(state.translation_buffer) > 30:
                state.translation_buffer.pop(0)
            
            if len(state.translation_buffer) < 30:
                info = f"📊 Acumulando: {len(state.translation_buffer)}/30"
            else:
                # Predecir
                sequence = np.array([state.translation_buffer])
                prediction = state.current_model.predict(sequence, verbose=0)
                predicted_class = np.argmax(prediction[0])
                confidence = prediction[0][predicted_class]
                
                sign_name = state.sign_map.get(predicted_class, f"Clase {predicted_class}")
                
                info = f"🔮 Seña: {sign_name.upper()}\\n"
                info += f"📊 Confianza: {confidence:.1%}\\n\\n"
                
                # Top 3
                top3_idx = np.argsort(prediction[0])[-3:][::-1]
                info += "Top 3:\\n"
                for i, idx in enumerate(top3_idx, 1):
                    name = state.sign_map.get(idx, f"Clase {idx}")
                    conf = prediction[0][idx]
                    info += f"  {i}. {name}: {conf:.1%}\\n"
    
    return processed_frame, info

# ============================================================================
# FUNCIONES DE CONTROL
# ============================================================================

def start_capture(sign_name, num_samples, frames_per_sample):
    """Inicia captura múltiple."""
    if not sign_name or not sign_name.strip():
        return "❌ Ingresa un nombre para la seña", gr.update(interactive=False)
    
    state.capturing = True
    state.current_sign = sign_name.strip().lower().replace(" ", "_")
    state.total_samples = num_samples
    state.frames_per_sample = frames_per_sample
    state.samples_captured = 0
    state.current_sequence = []
    
    return f"✅ Captura iniciada: '{state.current_sign}'\\n🎯 Meta: {num_samples} muestras\\n\\n▶️ Muestra tu mano...", gr.update(interactive=True)

def stop_capture():
    """Detiene captura."""
    was_capturing = state.capturing
    state.capturing = False
    
    if was_capturing:
        return f"⏹️ Captura detenida\\n📊 Capturadas: {state.samples_captured}/{state.total_samples}"
    return "⏹️ No había captura activa"

def load_model_for_translation(model_name):
    """Carga modelo para traducción."""
    try:
        if model_name == "No hay modelos":
            return "❌ No hay modelos entrenados"
        
        model_path = MODELS_DIR / model_name
        if not model_path.exists():
            return f"❌ Modelo no encontrado: {model_path}"
        
        state.current_model = tf.keras.models.load_model(str(model_path))
        
        # Cargar sign_map
        sign_map_path = DATA_DIR / "processed" / "sign_map.json"
        if sign_map_path.exists():
            with open(sign_map_path) as f:
                state.sign_map = {int(k): v for k, v in json.load(f).items()}
        
        state.translation_buffer = []
        
        return f"✅ Modelo cargado: {model_name}\\n📊 Señas: {len(state.sign_map)}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def train_model(epochs, batch_size, use_augmentation, progress=gr.Progress()):
    """Entrena el modelo."""
    if not TRAINING_AVAILABLE:
        return "❌ Módulos de entrenamiento no disponibles\\n\\nAgrega a tu repo:\\n- advanced_lstm_model.py\\n- training.py\\n\\nCódigo en: ALL_FILES_COMPLETE_CODE.md"
    
    try:
        progress(0, desc="Inicializando...")
        
        trainer = Trainer(
            data_dir=DATA_DIR,
            models_dir=MODELS_DIR,
            use_augmentation=use_augmentation
        )
        
        progress(0.1, desc="Cargando datos...")
        X, y, sign_map = trainer.load_and_prepare_data()
        
        if len(X) == 0:
            return "❌ No hay datos. Captura señas primero (Tab 1)."
        
        state.sign_map = sign_map
        
        progress(0.2, desc="Normalizando...")
        X = trainer.normalize_sequence_lengths(X, target_length=30)
        
        progress(0.3, desc="Dividiendo...")
        X_train, X_val, X_test, y_train, y_val, y_test = trainer.split_data(X, y)
        
        if use_augmentation:
            progress(0.4, desc="Augmentation...")
            X_train, y_train = trainer.augment_data(X_train, y_train)
        
        progress(0.5, desc="Entrenando...")
        model, history = trainer.train_model(
            X_train, y_train,
            X_val, y_val,
            num_classes=len(sign_map),
            epochs=epochs,
            batch_size=batch_size,
            model_name="sign_language_model"
        )
        
        state.current_model = model
        
        progress(0.9, desc="Evaluando...")
        results = trainer.evaluate_model(model, X_test, y_test, sign_map)
        
        progress(1.0, desc="Completado")
        
        output = f"✅ ENTRENAMIENTO COMPLETADO\\n\\n"
        output += f"📊 Dataset:\\n"
        output += f"   Señas: {len(sign_map)}\\n"
        output += f"   Train: {len(X_train)}\\n"
        output += f"   Val: {len(X_val)}\\n"
        output += f"   Test: {len(X_test)}\\n\\n"
        output += f"📈 Resultados:\\n"
        output += f"   Accuracy: {results['metrics']['accuracy']:.2%}\\n"
        output += f"   Top-3: {results['metrics'].get('top_3_accuracy', 0):.2%}\\n\\n"
        output += f"💾 Modelo: sign_language_model_best.h5\\n"
        
        return output
        
    except Exception as e:
        return f"❌ Error: {str(e)}"

def show_dataset_info():
    """Muestra info del dataset."""
    raw_dir = DATA_DIR / "raw"
    if not raw_dir.exists():
        return "📭 No hay datos capturados"
    
    sign_dirs = [d for d in raw_dir.iterdir() if d.is_dir()]
    
    if not sign_dirs:
        return "📭 No hay señas capturadas"
    
    output = f"📊 DATASET DE SEÑAS\\n" + "="*50 + "\\n\\n"
    output += f"🎯 Total de señas: {len(sign_dirs)}\\n\\n"
    
    total_samples = 0
    
    for sign_dir in sorted(sign_dirs):
        samples = list(sign_dir.glob("*.npy"))
        num_samples = len(samples)
        total_samples += num_samples
        
        output += f"📁 {sign_dir.name}: {num_samples} muestras"
        
        if num_samples < 10:
            output += " ⚠️\\n"
        elif num_samples >= 15:
            output += " ✅\\n"
        else:
            output += " ✓\\n"
    
    output += "\\n" + "="*50 + "\\n"
    output += f"📈 Total: {total_samples} muestras\\n"
    output += f"📊 Promedio: {total_samples / len(sign_dirs):.1f}/seña\\n"
    
    if len(sign_dirs) < 5:
        output += "\\n💡 Captura al menos 5 señas diferentes\\n"
    if total_samples < 75:
        output += "💡 Necesitas ~75 muestras (15×5)\\n"
    
    return output

def get_model_list():
    """Lista modelos disponibles."""
    models = [m.name for m in MODELS_DIR.glob("*.h5")]
    return models if models else ["No hay modelos"]

# ============================================================================
# INTERFAZ
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(), title="Sign Language Translator") as demo:
    
    gr.Markdown("""
    # 🤟 Sign Language Translator
    ### ✨ Procesamiento en Tiempo Real Unificado
    
    **Webcam + Keypoints sincronizados • Captura • Entrenamiento • Traducción**
    """)
    
    with gr.Tabs():
        
        # ====================================================================
        # TAB 1: CAPTURA
        # ====================================================================
        
        with gr.Tab("📹 Captura"):
            gr.Markdown("""
            ### Captura con procesamiento en tiempo real
            
            **La webcam y los keypoints están sincronizados en una sola vista**
            """)
            
            with gr.Row():
                # Webcam con procesamiento en tiempo real
                webcam_capture = gr.Image(
                    sources=["webcam"],
                    streaming=True,
                    label="Webcam + Keypoints (Tiempo Real)",
                    type="numpy"
                )
                
                # Info en tiempo real
                capture_info = gr.Textbox(
                    label="Estado",
                    lines=12,
                    max_lines=12
                )
            
            with gr.Row():
                sign_name_input = gr.Textbox(
                    label="Nombre de la Seña",
                    placeholder="Ej: hola, gracias, adios"
                )
                num_samples_input = gr.Slider(1, 30, 15, step=1, label="Muestras")
                frames_input = gr.Slider(15, 45, 30, step=5, label="Frames/Muestra")
            
            with gr.Row():
                start_btn = gr.Button("▶️ Iniciar Captura", variant="primary")
                stop_btn = gr.Button("⏹️ Detener", variant="stop")
            
            # Stream processing unificado
            webcam_capture.stream(
                fn=lambda frame: process_frame_unified(frame, mode="capture"),
                inputs=webcam_capture,
                outputs=[webcam_capture, capture_info],
                stream_every=0.1,
                time_limit=None
            )
            
            start_btn.click(
                fn=start_capture,
                inputs=[sign_name_input, num_samples_input, frames_input],
                outputs=[capture_info, start_btn]
            )
            
            stop_btn.click(
                fn=stop_capture,
                outputs=capture_info
            )
            
            gr.Markdown("""
            **💡 Tips:**
            - La vista es única: webcam + keypoints en tiempo real
            - Auto-pausa cuando no detecta manos
            - Captura 15-20 muestras por seña
            - Varía ángulos entre muestras
            """)
        
        # ====================================================================
        # TAB 2: ENTRENAMIENTO
        # ====================================================================
        
        with gr.Tab("🎓 Entrenamiento"):
            gr.Markdown("### Entrenar modelo Bi-LSTM + Attention")
            
            with gr.Row():
                epochs_input = gr.Slider(10, 200, 100, step=10, label="Épocas")
                batch_size_input = gr.Slider(8, 64, 16, step=8, label="Batch Size")
            
            augmentation_check = gr.Checkbox(
                label="Data Augmentation (Recomendado)",
                value=True
            )
            
            train_btn = gr.Button("🚀 Entrenar", variant="primary")
            train_output = gr.Textbox(label="Resultados", lines=15)
            
            train_btn.click(
                fn=train_model,
                inputs=[epochs_input, batch_size_input, augmentation_check],
                outputs=train_output
            )
            
            if not TRAINING_AVAILABLE:
                gr.Markdown("""
                ⚠️ **Módulos de entrenamiento no disponibles**
                
                Agrega a tu repo: `advanced_lstm_model.py` y `training.py`
                
                Código en: `ALL_FILES_COMPLETE_CODE.md`
                """)
        
        # ====================================================================
        # TAB 3: TRADUCCIÓN
        # ====================================================================
        
        with gr.Tab("🔮 Traducción"):
            gr.Markdown("""
            ### Traducción en tiempo real
            
            **Vista unificada: webcam + keypoints + predicción**
            """)
            
            model_selector = gr.Dropdown(
                label="Modelo",
                choices=get_model_list(),
                interactive=True
            )
            
            load_btn = gr.Button("📥 Cargar Modelo")
            load_status = gr.Textbox(label="Estado", lines=2)
            
            with gr.Row():
                # Webcam con procesamiento en tiempo real
                webcam_translate = gr.Image(
                    sources=["webcam"],
                    streaming=True,
                    label="Webcam + Keypoints + Predicción (Tiempo Real)",
                    type="numpy"
                )
                
                # Predicción en tiempo real
                translate_info = gr.Textbox(
                    label="Predicción",
                    lines=12,
                    max_lines=12
                )
            
            load_btn.click(
                fn=load_model_for_translation,
                inputs=model_selector,
                outputs=load_status
            )
            
            # Stream processing unificado
            webcam_translate.stream(
                fn=lambda frame: process_frame_unified(frame, mode="translate"),
                inputs=webcam_translate,
                outputs=[webcam_translate, translate_info],
                stream_every=0.1,
                time_limit=None
            )
            
            demo.load(
                fn=lambda: gr.Dropdown(choices=get_model_list()),
                outputs=model_selector
            )
        
        # ====================================================================
        # TAB 4: DATASET
        # ====================================================================
        
        with gr.Tab("📊 Dataset"):
            gr.Markdown("### Información del Dataset")
            
            refresh_btn = gr.Button("🔄 Actualizar")
            dataset_info = gr.Textbox(label="Dataset", lines=20)
            
            refresh_btn.click(fn=show_dataset_info, outputs=dataset_info)
            demo.load(fn=show_dataset_info, outputs=dataset_info)
    
    gr.Markdown("""
    ---
    ### 📚 Información
    
    **🎥 Mejora clave**: Procesamiento unificado en tiempo real
    - Una sola vista: webcam + keypoints sincronizados
    - No hay delay entre captura y procesamiento
    - Feedback instantáneo
    
    **💾 Datos**: `/content/drive/MyDrive/SignLanguageTranslator/`
    """)

print("\\n🎨 Iniciando interfaz con procesamiento unificado...\\n")
print("✅ Mejoras:")
print("   • Webcam + keypoints en una sola vista")
print("   • Procesamiento en tiempo real sincronizado")
print("   • Sin delay entre captura y visualización\\n")

demo.launch(debug=True, share=True)

---
# 📝 Notas

## ✅ Mejoras Implementadas

### 1. Webcam en Gradio
- `gr.Image(sources=["webcam"], streaming=True)`
- No necesita JavaScript de Colab
- Se ve directamente en la interfaz

### 2. Captura Completa
- Preview en vivo
- Captura múltiple automática
- Detección de manos con auto-pausa
- Feedback en tiempo real

### 3. Entrenamiento Funcional
- Carga datos del dataset
- Data augmentation
- Entrena Bi-LSTM + Attention
- Guarda mejor modelo

### 4. Traducción en Tiempo Real
- Carga modelos entrenados
- Traduce mientras muestras señas
- Top-3 predicciones
- Muestra confianza

## 🎯 Flujo Completo

```
1. CAPTURA (Tab 1)
   ├─ Click "Preview" → Ver mano + keypoints
   ├─ Ingresar nombre: "hola"
   ├─ Muestras: 15
   ├─ Click "Iniciar Captura"
   ├─ Mostrar mano → Graba automáticamente
   ├─ Sin mano → Auto-pausa
   └─ Repetir para 5+ señas

2. ENTRENAMIENTO (Tab 2)
   ├─ Épocas: 100
   ├─ Batch: 16
   ├─ Augmentation: ✅
   ├─ Click "Entrenar"
   └─ Esperar ~10-15 min

3. TRADUCCIÓN (Tab 3)
   ├─ Seleccionar modelo
   ├─ Click "Cargar Modelo"
   ├─ Mostrar seña a webcam
   └─ Ver predicción en tiempo real

4. DATASET (Tab 4)
   └─ Ver estadísticas y progreso
```

## ⚠️ Importante

**Para entrenamiento completo necesitas:**
- `src/advanced_lstm_model.py`
- `src/training.py`

**Código en**: `ALL_FILES_COMPLETE_CODE.md`

**Si no los tienes**, la captura y traducción funcionarán pero entrenamiento mostrará mensaje de error.

---

**¡Disfruta tu Sign Language Translator completo! 🤟**